In [29]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
raw_data = pd.read_json("pizza_data.json")
df = pd.DataFrame(raw_data)
df.head()

,Size,Cheese,Sauce,Crust,Popularity,Price,Comments,Pizza_ID,topping_1,topping_2,topping_3
0,Extra Large,No,Yes,Thin Crust,3 Stars,27.90,Sample comment,2,Black Olives,Sausage,Perpperoni
1,Large,No,Yes,Stuffed Crust,5 Stars,16.48,Sample comment,3,Spinach,None,None
2,Large,No,No,Greek Style,5 Stars,17.27,Sample comment,4,Red Peppers,Spinach,Hot Oil
3,Large,No,No,Deep Dish,1 Star,10.33,Sample comment,5,Green Peppers,None,None
4,Medium,No,No,Greek Style,4 Stars,13.19,Sample comment,6,Hot Oil,Mushrooms,Pineapple


In [3]:
 pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [33]:
features = ['Size', 'Cheese', 'Sauce', 'Crust', 'Popularity', 'Price', 'Pizza_ID', 'topping_1', 'topping_2', 'topping_3']

X = df[features]
X_encoded = pd.get_dummies(X)

nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X_encoded)
distances, indices  = nbrs.kneighbors(X_encoded)

print(indices[5])

[5 2 7 1]


In [66]:
def getNeighbors(pizza_id):
    neighbors = indices[pizza_id]
    new_neighbors = np.delete(neighbors, 0)
    print(new_neighbors)
    result = df.query('Pizza_ID in @new_neighbors')
    result.reset_index()
    display(result)
    json_file = "recs_results.json"
    result.to_json(json_file, orient="records")
        
getNeighbors(56)


[55 54 57]


,Size,Cheese,Sauce,Crust,Popularity,Price,Comments,Pizza_ID,topping_1,topping_2,topping_3
52,Large,Yes,No,Gluten Free,4 Stars,20.48,Sample comment,54,None,None,None
53,Small,Yes,No,Sicilian,3 Stars,16.78,Sample comment,55,Mushrooms,None,None
55,Large,Yes,No,Deep Dish,4 Stars,21.78,Sample comment,57,Red Peppers,None,None


In [67]:
import pickle

# Specify the file path to save the pipeline
file_path = 'trained_model.pkl'

# Save the pipeline to disk
with open(file_path, 'wb') as file:
    pickle.dump(nbrs, file)

In [69]:
with open('trained_model.pkl', 'rb') as file:
    model = pickle.load(file)

distances, indices  = model.kneighbors(X_encoded)



[5 2 7 1]
